# 스케줄 간격

- Airflow에서 DAG 일정 시간 간격으로 실행하기
- 시간, 일, 월 등
- schedule_interval

In [ ]:
# 하루에 한 번 DAG 실행하기
dag = DAG(
    dag_id="listing_2_10",
    start_date=airflow.utils.dates.days_ago(14), # 14일 전부터 시작 
    schedule_interval="@daily", # 워크플로 하루에 한 번 실행
)

# Airflow 태스크 실패

- 외부 서비스 중단, 네트워크 연결 문제, 디스크 손상 등
- 로그를 열어서 스택 확인 후 잠재적 문제 원인 확인
- 문제 해결 후 실패한 시점부터 다시 태스크 시작(전체 워크프로우 재실행 X)
- 실패한 태스크 클릭 후 팝업에서 'Clear' 버튼 클릭

# 스케줄링 사용자 이벤트 처리 예시

- 웹사이트에서 사용자 동작 추적 및 웹사이트에서 액세스한 페이지 분석 서비스 가정

In [ ]:
# API(로컬) 생성
# 30일 동안 모든 이벤트 목록 반환
# 사용자 통계를 계산하기 위해 분석할 수 있는 사용자 이벤트 목록 반환
"curl -o /data/events.json http://events_api:5000/events"

## unscheduled DAG
- BashOperator : 사용자 이벤트 데이터
- PythonOperator : 데이터 로드 및 이벤트 개수 확인
- UI, API 통해서 수동으로 트리거해서 실행

In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="01_unscheduled", 
    start_date=datetime(2019, 1, 1), # DAG 시작 날짜
    schedule_interval=None # 스케줄 X
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json http://events_api:5000/events" # API에서 이벤트 가져온 후 저장 
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    Path(output_path).parent.mkdir(exist_ok=True) # 출력 디렉터리 확인 

    events = pd.read_json(input_path) # 이벤트 데이터 로드 
    stats = events.groupby(["date", "user"]).size().reset_index() # 필요 통계 계산

    stats.to_csv(output_path, index=False) #  csv 파일 저장


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats # 실행 순서 

## daily_schedule DAG
- 매일 자정에 DAG 실행
- 정의된 간격 후에 태스크 시작
- 처음 실행 시간은 1월 2일. 1월 1일에는 실행되지 않음

In [ ]:
from datetime import datetime
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="02_daily_schedule",
    schedule_interval="@daily", # 매일 자정에 실행 
    start_date=datetime(2019, 1, 1), # 스케줄 시작 날짜/시간
    end_date=datetime(2019, 1, 5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json http://events_api:5000/events"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats

## with_end_date DAG
- DAG 종료 날짜 정의

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="03_with_end_date",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json http://events_api:5000/events"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats

## Cron 기반의 스케줄 간격 

- cron : macOS, 리눅스 등과 같은 유닉스 기반 OS에서 사용하는 시간 기반 작업 스케줄러
- ""* * * * *""
- 분 시간 일 월 요일 
- 0 * * * * : 매시간 정시에 실행
- 0 0 * * * : 매일 자정에 실행
- 0 0 * * 0 : 매주 일요일 자정에 실행
- 0 0 1 * * : 매월 1일 자정
- 45 23 * * SAT : 매주 토요일 23시 45분
- 0 0 * * MON, WED, FRI : 매주 월, 화, 금요일 자정에 실행
- 0 0 * * MON-FRI : 매주 월요일부터 금요일 자정에 실행
- 0 0,12 * * * : 매일 자정 및 오후 12시에 실행
- 특정 빈도마다 스케줄 정의 불가(ex. 3일에 한 번씩)

## 빈도 기반의 스케줄 간격

- dateitme 모듈에 포함된 timedelta 인스턴스 사용

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="04_time_delta",
    schedule_interval=dt.timedelta(days=3), # 시작 시간으로부터 3일마다 실행
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json http://events_api:5000/events"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats


# 데이터 증분 처리

- 데이터 순차적으로 가져올 수 있도록 DAG 변경
- 스케줄 간격에 해당하는 일자의 이벤트만 로드하고 새로운 이벤트만 통계 계산
- 데이터의 양 크게 줄여 훨씬 효율적
- 날짜별로 분리된 단일 파일로 저장해 매일 순차적으로 파일 저장 가능

- 워크플로에서 증분 데이터 처리를 구현하려면 DAG 수정하여 특정 날짜의 데이터 다운로드
- 시작 및 종료 날짜 매개변수 함께 정의

## query_with_dates DAG
- 이벤트 데이터의 시간 범위 매개변수
- end date는 포함하지 않는 날짜

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="05_query_with_dates",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json "
        "http://events_api:5000/events?"
        "start_date=2019-01-01&"
        "end_date=2019-01-02"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats

## templated_query DAG
- 2019-01-01 이외의 날짜에 대한 데이터 가져오기
- 실행 날짜 사용
- " : DAG가 실행되는 날짜와 시간을 나타내는 매개변수(스케줄 간격으로 실행되는 시작 시간)
- next_execution_date : 스케줄 간격의 종료 시간
- previous_execution : 과거의 스케줄 간격의 시작 정의

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="06_templated_query",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json "
        "http://events_api:5000/events?"
        "start_date={{execution_date.strftime('%Y-%m-%d')}}&" # Jinja 템플릿으로 형식화된 execution_date 삽입
        "end_date={{next_execution_date.strftime('%Y-%m-%d')}}" # next_execution_date로 다음 실행 간격 날짜 정의
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats


## templated_query_ds DAG
- 축약 매개변수 제공
- ds 및 ds_nodash 매개 변수는 각각 YYYY-MM-DD 및 YYYYMMDD 형식으로 된 execution_date의 다른 표현
- next_ds, next_ds_nodash
- prev_ds, prev_ds_nodash

In [ ]:
import datetime as dt
from datetime import timedelta
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="07_templated_query_ds",
    schedule_interval=timedelta(days=3),
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events.json "
        "http://events_api:5000/events?"
        "start_date={{ds}}&" # YYYY-MM_DD 형식의 execution_date 제공 
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(input_path, output_path):
    """Calculates event statistics."""

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    op_kwargs={"input_path": "/data/events.json", "output_path": "/data/stats.csv"},
    dag=dag,
)

fetch_events >> calculate_stats


## templated_path DAG
- 데이터 파티셔닝
- 파티셔닝 : 데이터 세트를 더 작고 관리하기 쉬운 조각으로 나누는 작업 
- 파티션 : 데이터 세트의 작은 부분
- 새로운 fetch_events 태스크로 이벤트 데이터 새롭게 스케줄한 간격에 맞춰 점진적으로 가져올 때, 각각의 새로운 태스크가 전일의 데이터 덮어쓰지 않게 함 
- events.json 파일에 새 이벤트 추가할 수 있으나 전체 데이터 세트 로드하는 다운스트림 프로세스 작업 필요
- 태스크의 출력을 해당 실행 날짜 이름이 적힌 파일에 기록해 데이터 세트 일일 배치로 나누는 방식 사용
- 매일 사용자 이벤트에 대한 통계 계산시 전체 데이터 세트 로드 및 전체 이벤트 기록에 대한 통계 계산할 필요 없음
- 각 파티션에 대한 통계 효율적으로 계산 가능

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="08_templated_path",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events/{{ds}}.json " # 반환된 값이 템플릿 파일 이름에 기록
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(**context): # 모든 콘텍스트 변수 수신 
    """Calculates event statistics."""
    input_path = context["templates_dict"]["input_path"] # templates_dict 개체에서 템플릿 값 검색
    output_path = context["templates_dict"]["output_path"]

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    templates_dict={ # 매개변수 사용. 템플릿화해야 하는 모든 인수 전달
        "input_path": "/data/events/{{ds}}.json",
        "output_path": "/data/stats/{{ds}}.csv",
    },
    # Required in Airflow 1.10 to access templates_dict, deprecated in Airflow 2+.
    # provide_context=True,
    dag=dag,
)


fetch_events >> calculate_stats

## no_catchup DAG

- 시작 날짜, 스케줄 간격 및 종료 날짜의 세 가지 매개변수 사용하여 DAG 실행 시점 제어
- 임의의 시작 날짜로부터 스케줄 간격 정의 가능
- 과거의 시작 날짜부터 과거 간격 정의 가능
- Backfilling : 과거 데이터 세트 로드하거나 분석하기 위해 DAG의 과거 시점 지정 실행 
- 과거 시작 날짜를 지정하고 해당 DAG 활성화하면 현재 시간 이전에 과거 시작 이후의 모든 스케줄 간격 생성됨
- catchup 변수에 의해 제어되며 false로 설정하여 비활성 가능
- 가장 최근 스케줄 간격에 대해서만 실행

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="09_no_catchup",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
    catchup=False,
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events/{{ds}}.json "
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(**context):
    """Calculates event statistics."""
    input_path = context["templates_dict"]["input_path"]
    output_path = context["templates_dict"]["output_path"]

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    templates_dict={
        "input_path": "/data/events/{{ds}}.json",
        "output_path": "/data/stats/{{ds}}.csv",
    },
    dag=dag,
)


fetch_events >> calculate_stats


## non_atomic_send DAG
- 원자성(atomicity) : 모든 것이 완료되거나 완료되지 않아야 함. 나눌 수 없고 돌이킬 수 없는 일련의 데이터베이스와 같은 작업 
- 각 실행이 끝날 때마다 상위 10명의 사용자에게 이메일 발송 기능 추가
- 통계 계산 함수에 이메일 보내는 함수 추가
- _email_stats 함수 실패 시 통계 발송이 실패했음에도 불구하고 작업 성공한 것처럼 보임


- 멱등성(dempotency) : 동일한 입력으로 동일한 태스크를 여러 번 호출해도 결과에 효력이 없어야 함
- 입력 변경 없이 태스크 다시 실행해도 전체 결과 변경되지 않아야 함

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="10_non_atomic_send",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
    catchup=True,
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events/{{ds}}.json "
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(**context):
    """Calculates event statistics."""
    input_path = context["templates_dict"]["input_path"]
    output_path = context["templates_dict"]["output_path"]

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)

    _email_stats(stats, email="user@example.com") # csv에 작성 후 이메일 보내면 단일 기능에서 두 가지 작업 수행되어 원자성 깨짐


def _email_stats(stats, email):
    """Send an email..."""
    print(f"Sending stats to {email}...")


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    templates_dict={
        "input_path": "/data/events/{{ds}}.json",
        "output_path": "/data/stats/{{ds}}.csv",
    },
    dag=dag,
)

fetch_events >> calculate_stats

## atomic_send DAG
- 다수 개의 태스크로 분리하여 원자성 개선
- 이메일 발송 기능 별도 태스크로 분리
- 이메일 전송에 실패해도 calculate_stats 작업의 결과에 영향 주지 않음
- 강한 의존성이 발생할 경우 단일 태스크 내에서 두 작업을 모두 유지하여 하나의 일관된 태스크 단위를 형성하는 것이 더 나을 수 있음(로그인 후 이벤트 API 호출 등)
- 대부분의 Airflow 오퍼레이터는 이미 원자성을 유지하도록 설계됨  

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="11_atomic_send",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
    catchup=True,
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events/{{ds}}.json "
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(**context):
    """Calculates event statistics."""
    input_path = context["templates_dict"]["input_path"]
    output_path = context["templates_dict"]["output_path"]

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    templates_dict={
        "input_path": "/data/events/{{ds}}.json",
        "output_path": "/data/stats/{{ds}}.csv",
    },
    dag=dag,
)


def email_stats(stats, email):
    """Send an email..."""
    print(f"Sending stats to {email}...")


def _send_stats(email, **context):
    stats = pd.read_csv(context["templates_dict"]["stats_path"])
    email_stats(stats, email=email)


send_stats = PythonOperator(
    task_id="send_stats",
    python_callable=_send_stats,
    op_kwargs={"email": "user@example.com"},
    templates_dict={"stats_path": "/data/stats/{{ds}}.csv"},
    dag=dag,
)

fetch_events >> calculate_stats >> send_stats

## templated_path DAG (반복)

- 멱등성(dempotency) : 동일한 입력으로 동일한 태스크를 여러 번 호출해도 결과에 효력이 없어야 함
- 입력 변경 없이 태스크 다시 실행해도 전체 결과 변경되지 않아야 함

In [ ]:
import datetime as dt
from pathlib import Path

import pandas as pd

from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator

dag = DAG(
    dag_id="08_templated_path",
    schedule_interval="@daily",
    start_date=dt.datetime(year=2019, month=1, day=1),
    end_date=dt.datetime(year=2019, month=1, day=5),
)

fetch_events = BashOperator(
    task_id="fetch_events",
    bash_command=(
        "mkdir -p /data/events && "
        "curl -o /data/events/{{ds}}.json " # 템플릿 파일 이름을 설정하여 분할
        "http://events_api:5000/events?"
        "start_date={{ds}}&"
        "end_date={{next_ds}}"
    ),
    dag=dag,
)


def _calculate_stats(**context):
    """Calculates event statistics."""
    input_path = context["templates_dict"]["input_path"]
    output_path = context["templates_dict"]["output_path"]

    events = pd.read_json(input_path)
    stats = events.groupby(["date", "user"]).size().reset_index()

    Path(output_path).parent.mkdir(exist_ok=True)
    stats.to_csv(output_path, index=False)


calculate_stats = PythonOperator(
    task_id="calculate_stats",
    python_callable=_calculate_stats,
    templates_dict={
        "input_path": "/data/events/{{ds}}.json",
        "output_path": "/data/stats/{{ds}}.csv",
    },
    # Required in Airflow 1.10 to access templates_dict, deprecated in Airflow 2+.
    # provide_context=True,
    dag=dag,
)


fetch_events >> calculate_stats

# Airflow 콘텍스트 사용하여 태스크 템플릿 작업

- 페이지 뷰 증가는 긍정적임 감성 -> 회사 주식 증가 가능성
- 위키미디어 재단 2015년 이후의 모든 페이지 뷰 컴퓨터가 읽을 수 있는 형식ㅇ으로 제공
- 페이지 뷰는 gzip 형식으로 다운로드 가능. 시간당 페이지 뷰 수 집계

# Reference

1. 블로그(검색어)
- https://velog.io/@jewon119/01.Flask-%EA%B8%B0%EC%B4%88-Jinja-template(Jinja 템플릿)
- https://coding-grandpa.tistory.com/2 (파이썬 보일러플레이트)